# Base de données

## connexion

La méthode `connect` permet d’obtenir une connexion à la base de données

En fonction du sgbd, les paramètres peuvent varier mais le plus souvent :
- host : 		nom du serveur
- dbname 		nom de la base de données
- user 		utilisateur réalisant la connexion
- password  	mot de passe  

Apres utilisation, un connexion doit être fermé via la méthode `close`


### SQLLite

In [5]:
import sqlite3
conn = sqlite3.connect('bd.db')
conn.close()

### Postgres

In [17]:
import psycopg2
conn = psycopg2.connect(host="127.0.0.1",
                        dbname="demo",
                        user="user",
                        password="secret")


conn.close()

# Mysql

In [18]:

import mysql.connector as database
connection = database.connect(
    user="user",
    password="secret",
    host="localhost",
    database="demo")

connection.close()


>> creation de la table PERSONNE


# Création d'une table

In [4]:
import sqlite3
conn = sqlite3.connect('bd.db')

cursor = conn.cursor()
cursor.execute("""
            create table IF NOT EXISTS PERSONNE(
                id INTEGER  PRIMARY KEY  AUTOINCREMENT,
                nom VARCHAR(100),
                prenom VARCHAR(100),
                date_naissance VARCHAR(10)
            )
            """)
conn.close()

## Insérer des donnnées

In [28]:
import sqlite3
conn = sqlite3.connect('bd.db')

cursor = conn.cursor()
cursor.execute("delete from personne")
cursor.execute("""
                insert into personne(nom, prenom, date_naissance) values (
                :nom, :prenom,:date_naissance);
            """, guido)


cursor.execute("""
                insert into personne(nom, prenom, date_naissance) values (
                ?, ?, ?);
            """, ("James", "Gosling", "19/06/1955"))

conn.commit()
conn.close()

## Récupérer des données

Exécuter la requête de sélection en utilisant la méthode « execute  »

Pour récupérer les données, il faut utiliser une des méthodes suivantes : 
- `fetchone` pour récupérer un seul enregistrement
- `fetchall` pour récupérer un ensemble de lignes
- `fetchmany` : resemble à la méthode fecthall sauf que la méthode ne charge en mémoire qu’un ensemble des résultats


In [30]:
conn = sqlite3.connect('bd.db')

cursor = conn.cursor()

cursor.execute("select * from personne")
rs = cursor.fetchall()
print(*rs, sep="\n")

conn.close()


(1, 'Guido', 'van Rossum', '31/01/1956')
(2, 'James', 'Gosling', '19/06/1955')


On constate que le résultat est retourné sous la forme d'un tableau ne contenant uniquement que les données. Il est souvent préférable de récupérer le nom des colonnes associé à chaque données.

Il suffit d'indiquer la factory pour le curseur.

In [39]:
conn = sqlite3.connect('bd.db')
cursor = conn.cursor()
cursor.row_factory = sqlite3.Row

cursor.execute("select * from personne")
rs = cursor.fetchall()
for result in rs:
    for key in result.keys():
        print("{} : {}".format(key, result[key]))
    print("-------")

conn.close()

id : 1
nom : Guido
prenom : van Rossum
date_naissance : 31/01/1956
-------
id : 2
nom : James
prenom : Gosling
date_naissance : 19/06/1955
-------


## Gestion des transactions

En cas d’erreur, il est nécessaire d’invalider la transaction. 

La méthode « rollback »permet d’annuler les requêtes d’insertion, de mise à jour ou de suppression exécuter sur le SGBD

En utilisant le mot clé try / except / finally


In [40]:
conn = None
try:
    conn = sqlite3.connect('bd.db')
    cursor = conn.cursor()
    cursor.execute("""
                insert into personne(nom, prenom, date_naissance) values (
                ?, ?, ?);
            """, ("Ritchie", "Dennis", "08/09/19841"))
    conn.commit()
except sqlite3.OperationalError as e:
    print("---- erreur detectée ----", e)
    conn.rollback()
finally:
    if conn != None:
        conn.close()

## Le mot clé `with`

Le mot clé with intervient dans la gestion des ressources afin de garantir que la ressource est correctement libérée

Pour la connexion, le mot with permettra de gérer automatiquement la connexion

Pour le curseur,  le mot with fermera automatiquement le curseur (en fonction du sgbd)


In [44]:
conn = None
with sqlite3.connect('bd.db') as conn:
    cursor = conn.cursor()
    cursor.execute("""
                   insert into personne(nom, prenom, date_naissance) values (
                   ?, ?, ?);
               """, ("Ritchie", "Dennis", "08/09/19841"))

conn.close()